In [ ]:
# default_exp uvis
# default_cls_lvl 3

# UVIS
> Data management for Cassini UVIS.

In [ ]:
#hide
from nbverbose.showdoc import show_doc  # noqa

In [ ]:
# export
import tomlkit
from yarl import URL

from planetarypy.config import config
from planetarypy.pds.opusapi import OPUS
from planetarypy.utils import url_retrieve

In [ ]:
# export
storage_root = config.storage_root / "missions/cassini/uvis"
storage_root

Path('/home/maye/big_drive/planetary_data/missions/cassini/uvis')

In [ ]:
get_user_guide()

Path('/home/maye/big_drive/planetary_data/missions/cassini/uvis/uvis_user_guide.pdf')

In [ ]:
# export
class DataManager:
    def __init__(
        self, 
        pid: str,  # Product ID. If longer than PDS_ID, will be cut in attribute `pds_id` 
    ):
        self.pid = pid
        self.dict = None
        if not self.raw_data_path.exists():
            self.download()

    def query(self, pds_id=None):
        pds_id = pds_id if pds_id is not None else self.pds_id
        opus = OPUS(silent=True)
        try:
            self.query_result = opus.query_image_id(pds_id)[0]
        except IndexError:
            raise FileNotFoundError("Project ID not found on PDS server.")
        self.opus_id = self.query_result[0]
        self.dict = self.query_result[1]

    @property
    def pds_id(self):
        return self.pid[:17]

    @property
    def folder(self):
        #         return storage_root / "/".join(self.raw_data_url.parts[4:7])
        return storage_root / self.pds_id

    @property
    def raw_data_url(self):
        if not self.dict:
            self.query()
        return URL(self.dict["couvis_raw"][0])

    @property
    def raw_label_url(self):
        if not self.dict:
            self.query()
        return URL(self.dict["couvis_raw"][1])

    @property
    def raw_data_path(self):
        return self.folder / (self.pds_id + ".DAT")

    @property
    def raw_label_path(self):
        return self.raw_data_path.with_suffix(".LBL")

    @property
    def calib_corr_path(self):
        return self.raw_data_path.with_name(self.raw_data_path.stem + "_CAL_3.DAT")

    @property
    def calib_label_path(self):
        return self.calib_corr_path.with_suffix(".LBL")

    @property
    def original_pid_file(self):
        return self.folder / "original_pid.txt"

    @property
    def results_file(self):
        return self.folder / "urls.toml"

    def download(self, overwrite=False):
        if self.raw_data_path.exists() and not overwrite:
            print("Local files exists. Use `overwrite=True` to download fresh.")
            return
        self.query()
        self.original_pid_file.mk_write(self.pid)
        self.results_file.mk_write(tomlkit.dumps(self.dict))
        self.raw_data_path.parent.mkdir(parents=True, exist_ok=True)
        for key in ["couvis_raw", "couvis_calib_corr"]:
            for url in self.dict[key]:
                url_retrieve(url, self.folder / URL(url).name)

    def __repr__(self):
        s = f"Product ID:\n{self.id}\n\n"
        for k, v in self.query_result[1].items():
            s += f"Key: {k},\nValue(s):\n{v}\n\n"
        return s

In [ ]:
pid = "EUV2002_198_03_26_54_UVIS_C33ST_SPICARAST002_PRIME"

In [ ]:
dm = DataManager(pid)

In [ ]:
dm.raw_label_url

URL('https://opus.pds-rings.seti.org/holdings/volumes/COUVIS_0xxx/COUVIS_0004/DATA/D2002_198/EUV2002_198_03_26.LBL')

In [ ]:
assert dm.pid == pid

PDS data is only stored under the first 17 characters of any UVIS-internal product id.

In [ ]:
assert dm.pds_id == pid[:17]

The originally used product ID `pid` will be stored under `original_pid.txt` in the data folder:

In [ ]:
dm.original_pid_file

Path('/home/maye/big_drive/planetary_data/missions/cassini/uvis/EUV2002_198_03_26/original_pid.txt')

Also the OPUS result file will be stored as a TOML file:

In [ ]:
dm.results_file

Path('/home/maye/big_drive/planetary_data/missions/cassini/uvis/EUV2002_198_03_26/urls.toml')

In [ ]:
dm.download(overwrite=False)

Local files exists. Use `overwrite=True` to download fresh.


In [ ]:
dm.raw_data_path

Path('/home/maye/big_drive/planetary_data/missions/cassini/uvis/EUV2002_198_03_26/EUV2002_198_03_26.DAT')

In [ ]:
dm.calib_corr_path

Path('/home/maye/big_drive/planetary_data/missions/cassini/uvis/EUV2002_198_03_26/EUV2002_198_03_26_CAL_3.DAT')

In [ ]:
# export
def get_data_path(pid):
    dm = DataManager(pid)
    return dm.raw_data_path


def get_label_path(pid):
    dm = DataManager(pid)
    return dm.raw_label_path

In [ ]:
get_data_path(pid)

Path('/home/maye/big_drive/planetary_data/missions/cassini/uvis/EUV2002_198_03_26/EUV2002_198_03_26.DAT')

In [ ]:
get_label_path(pid)

Path('/home/maye/big_drive/planetary_data/missions/cassini/uvis/EUV2002_198_03_26/EUV2002_198_03_26.LBL')

In [ ]:
# export
def get_user_guide():
    url = URL(
        "https://pds-rings.seti.org/cassini/uvis/1-UVIS_Users_Guide_-2018-Jan%2015-For%20PDS-REV-2018-07-06.pdf"
    )
    local_path = storage_root / "uvis_user_guide.pdf"
    if not local_path.exists():
        url_retrieve(url, storage_root / "uvis_user_guide.pdf")
    return local_path